# Marketing Analytics: Data Enrichment Pipeline

## Overview
This notebook documents the ETL (Extract, Transform, Load) process for the Marketing Analytics dashboard. 
We are transforming a Star Schema (Fact + Dimensions) into a single wide Master Table for Tableau.

### Data Architecture
* **Fact Table**: `fact_performance.csv` (Transactions)
* **Dimensions**: 
    * `dim_campaign.csv` (Campaign Context)
    * `dim_source.csv` (Channel/Platform Context)
    * `dim_date.csv` (Time Context)

In [ ]:
import pandas as pd
import numpy as np

## 1. Data Extraction
Loading the raw CSV files into pandas DataFrames.

In [ ]:
# Load the datasets
fact_performance = pd.read_csv('fact_performance.csv')
dim_campaign = pd.read_csv('dim_campaign.csv')
dim_source = pd.read_csv('dim_source.csv')
dim_date = pd.read_csv('dim_date.csv')

# Verification
print(f"Fact Table Size: {fact_performance.shape}")

## 2. Data Transformation: Merging
Joining the Fact table with Dimensions to create a unified view.

* **Campaign Info**: Joined on `campaign_id`
* **Source Info**: Joined on `source_id`
* **Date Info**: Joined on `date_key`

In [ ]:
# Left join fact table with dimensions
df_master = fact_performance.merge(dim_campaign, on='campaign_id', how='left')
df_master = df_master.merge(dim_source, on='source_id', how='left')
df_master = df_master.merge(dim_date, on='date_key', how='left')

## 3. Feature Engineering: KPI Calculation
Calculating standard marketing metrics. We use `np.where` to handle potential division-by-zero errors safely.

### Formulas
* **CPM (Cost Per Mille)**: $\frac{\text{Spend}}{\text{Impressions}} \times 1000$
* **CTR (Click-Through Rate)**: $\frac{\text{Clicks}}{\text{Impressions}} \times 100$
* **CPC (Cost Per Click)**: $\frac{\text{Spend}}{\text{Clicks}}$
* **Conversion Rate**: $\frac{\text{Conversions}}{\text{Impressions}} \times 100$

In [ ]:
# CPM: (Spend / Impressions) * 1000
df_master['CPM'] = np.where(df_master['impressions'] > 0, 
                            (df_master['spend'] / df_master['impressions']) * 1000, 
                            0)

# CTR: (Clicks / Impressions) * 100
df_master['CTR'] = np.where(df_master['impressions'] > 0, 
                            (df_master['clicks'] / df_master['impressions']) * 100, 
                            0)

# CPC: Spend / Clicks
df_master['CPC'] = np.where(df_master['clicks'] > 0, 
                            df_master['spend'] / df_master['clicks'], 
                            0)

# Conversion Rate (Conversions / Impressions * 100)
df_master['Conversion_Rate'] = np.where(df_master['impressions'] > 0, 
                                        (df_master['conversions'] / df_master['impressions']) * 100, 
                                        0)

## 4. Quality Assurance & Export
Previewing the final dataset before exporting to CSV for visualization tools (Tableau/PowerBI).

In [ ]:
# Check for nulls or anomalies
print(df_master.info())

# Preview key metrics
df_master[['date', 'campaign_name', 'channel', 'spend', 'CPM', 'CTR', 'CPC', 'Conversion_Rate']].head()

In [ ]:
# Export to CSV
df_master.to_csv('marketing_analytics_master.csv', index=False)
print("Export complete: marketing_analytics_master.csv")